In [1]:
import matplotlib
import networkx as nx
from itertools import permutations, product, filterfalse

In [ ]:
TAG_5P = 79.9663
TAG_3OH = 18.0106

In [ ]:
def gap_fill(df, df_skeleton, fullmass_dot, orientation=5, major=False, start_at=0, stop_at=0):
    """Implementation of the GapFill algorithm.
    
    :param df: pandas DataFrame, the original dataset.
    :param df_skeleton: pandas DataFrame, the skeleton compounds of the ladder.
    :param fullmass_dot: pandas DataFrame, should only contains one row, the intact compound.
    :param orientation: the orientation of the ladder. 3 or 5.
    :param major: bool, indicates how to deal with gaps, if False, the function only handles 
    the gap with mass window less than 3200, otherwise, it deals with every gaps, please note
    that it may takes tons of time to deal with big gaps.
    :param start_at, stop_at: restrict the Mass range the function handles.
    """
    THRESHOLD = 10000
    if major:
        THRESHOLD = 3200
    df_skeleton = df_skeleton.sort_values('Mass')

    idxs = list()
    gap_idxs = list()
    for idx, row in df_skeleton.iterrows():
        if start_at > 0 and row.Mass < start_at:
            continue
        if stop_at > 0 and row.Mass > stop_at:
            break
        print('Processing {}'.format(row.Mass))
        columns = ['Mass', 'RT', 'Vol']
        df_ends = pd.DataFrame(index=[0, 1], columns=columns)
        if not idxs:
            # handle gap between TAG and the first dot
            pre = TAG_5P + TAG_3OH
            if orientation == 3:
                pre = TAG_3OH - TAG_5P
            df_ends.iloc[0] = [pre, 0.001, 0]
        else:
            pre = df_skeleton.loc[idxs[-1]].Mass
            df_ends.iloc[0] = df_skeleton.loc[idxs[-1]][columns]
        is_last = True if idx == df_skeleton.index[-1] else False
        if not is_last and row.Mass - pre < 600:
            continue
        if row.Mass - pre > THRESHOLD:
            idxs.append(idx)
            continue
#         print('components {}-{}'.format(row.Mass, pre))
        res = components(row.Mass - pre)
        if not res.empty:
            # keep this dot, and fill in the gap
            print('keep dot {}'.format(row.Mass))
            idxs.append(idx)
            df_ends.iloc[1] = row[columns]
            df_ends = df_ends.astype(float)
            df_alones = standalone_dots(df, df_ends, mode='all')
            df_alones = filter_dots_in_mid_gap(df_alones)
            if not df_alones.empty:
                print('extend dots ', df_alones)
                gap_idxs.extend(df_alones.index)

        if is_last and row.Mass - pre < THRESHOLD and not fullmass_dot.empty:
            df_ends.iloc[0] = df_skeleton.loc[idxs[-1]][columns]
            df_ends.iloc[1] = fullmass_dot.iloc[0][columns]
            if stop_at > 0 and df_ends.iloc[1].Mass > stop_at:
                break
            if orientation == 5:
                df_ends.iloc[1]['Mass'] += TAG_5P
            elif orientation == 3:
                df_ends.iloc[1]['Mass'] -= TAG_5P
            df_alones = standalone_dots(df, df_ends, mode='all')
            if not df_alones.empty:
                print('extend dots ', df_alones)
                gap_idxs.extend(df_alones.index)

            # append the fullmass dot
            gap_idxs.append(fullmass_dot.index[0])

    # remove duplicated dots from df_gap
    df_gap = df.loc[gap_idxs]
    df_gap_dup = df_gap[df_gap.index.isin(df_skeleton.index)]
    df_gap = df_gap.drop(df_gap_dup.index)
    return df_gap

In [ ]:
def _modifications_df():
    csv = '../statics/bases_methyl.csv'
    df_mod = pd.read_csv(csv)
    df_mod.rename(columns={'Exact Mass': 'Mass'}, inplace=True)
    dfm = df_mod.copy()
    return dfm

def _get_product(m, n):
    def filter_product(i):
        return sum(i) != m

    r = m if m < 20 else int(m // 2)
    l = product(range(r+1), repeat=n-1)
    l = list(filterfalse(filter_product, l))

    max_methyl = m if m < 8 else 8
    max_methyl = 3 if m < 3 else m
    products = list()
    for i in range(max_methyl):
        # allow a base has two methylations
        prod = [item+(i,) for item in l if sum(item[:4])>=i-1]
        products.extend(prod)
    return products

def _get_permutations(m, n):
    """return m**n permutations, where the number of G should be at least 1
    """
    l = list(product(range(m+5), repeat=n))
    l = [i for i in l if sum(i[:-1]) in range(m-1, m+2) and i[-1]<=m/2+1]
    l = np.array(l)
    return l

def _handle_bases(mass, df_mod):
    """given mass value, find out bases permutations that has sum value near to mass
    return DataFrame, each row contains a permutation of A/C/G/U/D/Methyl and their masses sum
    """
    size = round(mass/320)
    if 200 < mass < 320:
        size = 1
    elif size < 1:
        rounded_count = int(round(mass / df_mod.Mass.iloc[-1]))
        if abs(rounded_count  * df_mod.Mass.iloc[-1] - mass) < 0.2:
            df = pd.DataFrame(columns=df_mod.Name)
#             df.loc[0] = [0, 0, 0, 0, 0, rounded_count]
            df.loc[0, 'Methyl'] = rounded_count
            df['Mass'] = rounded_count * df_mod.Mass.iloc[-1]
            df.fillna(0, inplace=True)
            return df
        return pd.DataFrame()
#     if size > 20:
#         size = size // 2
    perms = _get_product(size, df_mod.shape[0])
    seq_masses = np.matmul(perms, np.array(df_mod.Mass))
    seq_masses_pd = pd.Series(seq_masses)
    res_masses = perms.copy()
    res_masses = pd.DataFrame(res_masses, columns=df_mod.Name)
    res_masses['Mass'] = seq_masses_pd
    return res_masses[res_masses.Mass <= mass+1]

def _calc_bass_perms_and_remainder(mass, df_mod):
    """given mass value
    return DataFrame, each row contains the permutation of A/C/G/U and 
    mass diff between their masses sum and given mass
    """
    seq_masses = _handle_bases(mass, df_mod)
    if seq_masses.empty:
        return pd.DataFrame()

    diff = seq_masses
    diff['MassDiff'] = mass - seq_masses['Mass']
    valid_diff = diff[(diff.MassDiff>-0.1) & (diff.MassDiff<0.1)]
    return valid_diff

def components(mass):
    mass = abs(mass)
    if mass > 10000:
        return pd.DataFrame()
    df_mod = _modifications_df()
    df_bpr = _calc_bass_perms_and_remainder(mass, df_mod)
    return df_bpr

In [ ]:
def gap_rect(df_ends, mode='all'):
    df_ends = df_ends.astype('float')
    z_zoomin = np.polyfit(df_ends.Mass, df_ends.RT, 1)
    def f_zoomin(x):
        return z_zoomin[0] * x + z_zoomin[1]

    df_zoomin = df_ends.sort_values('Mass')
    if mode == 'left':
        l = df_ends.iloc[0].Mass + 300
        r = df_ends.iloc[1].Mass
    elif mode == 'right':
        l = df_ends.iloc[0].Mass
        r = df_ends.iloc[1].Mass - 300
    else:
        l = df_ends.iloc[0].Mass + 200
        r = df_ends.iloc[1].Mass - 200
    df_zoomin = df_ends.sort_values('RT')
    b = f_zoomin(l-150) 
    t = f_zoomin(r+150)
    if l > 10000:
        b = b - 0.2
        t = t + 0.2
    return l, r, b, t

def all_dots_in_gap(df, df_ends, mode='all'):
    l, r, b, t = gap_rect(df_ends, mode)
    if mode == 'left':
        df_gap = df[(df.Mass > l) & (df.Mass < r) & (df.RT > b)]
    elif mode == 'right':
        df_gap = df[(df.Mass > l) & (df.Mass < r) & (df.RT < t)]
    else:
        df_gap = df[(df.Mass > l) & (df.Mass < r) & (df.RT > b) & (df.RT < t)]
    print(l, r, b, t)
    return df_gap

def standalone_dots_in_gap(df_gap, df_ends, mode='all'):
    idxs = list()
    for idx, row in df_gap.iterrows():
        delta_left = abs(row.Mass - df_ends.iloc[0]['Mass'])
        delta_right = abs(row.Mass - df_ends.iloc[1]['Mass'])
        df_res_left = components(delta_left)
        df_res_right = components(delta_right)
        
        if mode == 'left' and df_res_left.shape[0] > 0:
            idxs.append(idx)
        elif mode == 'right' and df_res_right.shape[0] > 0:
            idxs.append(idx)
        elif mode == 'all' and (df_res_left.shape[0] > 0 and df_res_right.shape[0] > 0):
            idxs.append(idx)
    
    return df_gap.loc[idxs]

def standalone_dots(df, df_ends, mode='all'):
    df_res_list = list()
    for i in range(0, df_ends.shape[0], 2):
        df_end = df_ends.iloc[i:i+2]
        df_gap = all_dots_in_gap(df, df_end, mode)
        print('Processing the gap {}-{}, {} dots'.format(df_end.iloc[0]['Mass'], df_end.iloc[1]['Mass'], df_gap.shape[0]))
        df_res = standalone_dots_in_gap(df_gap, df_end, mode)
        if not df_res.empty:
            df_res_list.append(df_res)

    if not df_res_list:
        print('No dots found in these gaps.')
        return pd.DataFrame()
    df_standalones = pd.concat(df_res_list)
    df_standalones.drop_duplicates(inplace=True)
    
    return df_standalones

In [ ]:
def soft_connected(mass_diff):
    if mass_diff <= 0:
        return False
    df = components(mass_diff)
    return not df.empty

comp_vec = np.vectorize(soft_connected)

def zip_list(G):
    if not G.nodes:
        return []
    node_count_list = [(node, len(G.edges(node)), G.nodes[node]['Vol']) for node in G.nodes]
    node_count_list.sort(key=lambda x: x[2])
    node_count_list.sort(key=lambda x: x[1])
    return node_count_list

def filter_dots_in_mid_gap(df):
    if df.empty:
        return pd.DataFrame()
    if df.shape[0] == 1:
        return df
    df_x = df.Mass.copy()
    df_y = df.Mass.copy()
    np_x = np.array(df_x)
    np_y = np.array(df_y)
    np_xy = np_x[:, np.newaxis] - np_y
    
    np_conns = comp_vec(np_xy)
    df_conns = pd.DataFrame(np_conns)

    idx_pairs = list(df_conns[df_conns == True].stack().index)
    if not idx_pairs:
        return pd.DataFrame()
    G = nx.Graph()
    for pair in idx_pairs:
        a = df.iloc[pair[0]].Mass
        a_vol = df.iloc[pair[0]].Vol
        b = df.iloc[pair[1]].Mass
        b_vol = df.iloc[pair[1]].Vol
        G.add_edge(a, b)
        G.nodes[a]['Vol'] = a_vol
        G.nodes[b]['Vol'] = b_vol

    while True:
        l = zip_list(G)
        if not l:
            break
        max_edges = l[-1][1]
        if l[0][1] >= max_edges:
            break
        G.remove_node(l[0][0])
    l = zip_list(G)
    masses = [node[0] for node in l]
    print('reserved masses: ', masses)
    return df[df.Mass.isin(masses)]